# Wide input

## 기존 : Type & Level
## NEW : watching status ( watched & HAVEN't watched ) & Type

- watched : IF watching status is 'Watching', 'Completed', 'On-Hold', 'Dropped'
- ELSE : IF watching status is 'Plan to Watch'

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Embedding, concatenate, Flatten, Dense, Dropout, merge, Activation
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import ReLU, LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## < 데이터 설명 >

- anime : 만화에 대한 상세정보 ( 장르, 상영시기, 제작사, 등)
- animelist : user별 평점, 시청상태, 본 회차 수
- anime_synop : 만화 줄거리
- rating_complete : animelist 중 모든 회차 시청 및 평점을 준 데이터

## < watching status >
- 1 : Currently Watching
- 2 : Completed
- 3 : On Hold
- 4 : Dropped
- 5 : Plan to Watch

In [4]:
anime_10 = pd.read_csv('/content/drive/Shareddrives/DSL_2021_2_추천시스템/추천시스템 발표구현/Wide & Deep/anime_merge_sampling_100000.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
anime_10 = anime_10.drop(columns=['Score-10','Score-9','Score-8','Score-7','Score-6','Score-5','Score-4','Score-3','Score-2','Score-1']).sort_values(by='user_id')


In [6]:
anime_10['Genres'] = anime_10['Genres'].str.split(', ').str[0]

In [7]:
# watched_genres = anime_genres.groupby(['user_id'])['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia',
#        'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai',
#        'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts',
#        'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
#        'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
#        'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life',
#        'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Unknown',
#        'Vampire', 'Yaoi', 'Yuri'].agg('sum')
# watched_genres.shape

In [8]:
data = anime_10
data[data=='Unknown'] = np.nan #Unknown값 nan으로 대체 후 dropna 실행
data.dropna(axis=0, inplace=True) 
userrating = data.groupby(["user_id"])["rating"].agg('mean').to_frame(name='avg_rating').reset_index()
userrating

,user_id,avg_rating
0,12,10.0
1,16,6.0
2,19,5.0
3,20,9.0
4,21,7.5
...,...,...
47069,353360,8.0
47070,353365,9.0
47071,353369,9.0
47072,353383,9.0


In [9]:
merged = pd.merge(data, userrating, on='user_id', sort='user_id')
high_rating = np.where(merged["rating"] >= merged["avg_rating"], 1, 0)
merged['high_rating'] = high_rating

In [10]:
merged['Episodes'] = merged['Episodes'].astype('int64') 

# 시청진행율 추가 및 'watched_episode제거
merged['watch_rate'] = merged['watched_episodes'] / merged['Episodes']
watched_30 = np.where(merged["watch_rate"]>= 0.3, 1,0)
merged['watched_30'] = watched_30
merged.drop('watched_episodes', axis=1, inplace=True)
merged

,Unnamed: 0,user_id,anime_id,rating,watching_status,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,avg_rating,high_rating,watch_rate,watched_30
0,1343,12,28121,10,2,28121,Dungeon ni Deai wo Motomeru no wa Machigatteir...,7.64,Action,Is It Wrong to Try to Pick Up Girls in a Dungeon?,ダンジョンに出会いを求めるのは間違っているだろうか,TV,13,"Apr 4, 2015 to Jun 27, 2015",Spring 2015,"Genco, Movic, Warner Bros. Japan, KlockWorx, S...",Sentai Filmworks,J.C.Staff,Light novel,24 min. per ep.,PG-13 - Teens 13 or older,1172.0,60,1075095,10275,41249,870168,14498,22709,126471,10.0,1,1.000000,1
1,1722,16,2993,6,2,2993,Rosario to Vampire,6.84,Harem,Rosario + Vampire,ロザリオとバンパイア,TV,13,"Jan 3, 2008 to Mar 27, 2008",Winter 2008,"GDH, Yomiko Advertising, Happinet Pictures, At...",Funimation,Gonzo,Manga,25 min. per ep.,R+ - Mild Nudity,4285.0,208,545750,3794,17783,423659,11157,24451,68700,6.0,1,1.000000,1
2,3285,19,15699,5,4,15699,Haiyore! Nyaruko-san W,7.28,Sci-Fi,Nyaruko:Crawling With Love! Second Season,這いよれ！ニャル子さん W,TV,12,"Apr 8, 2013 to Jul 1, 2013",Spring 2013,"Avex Entertainment, Sotsu, DIVE II Entertainment","NIS America, Inc.",Xebec,Light novel,24 min. per ep.,PG-13 - Teens 13 or older,2442.0,1010,147938,307,5869,104404,3528,3520,30617,5.0,1,0.083333,0
3,3440,20,39597,9,2,39597,Sword Art Online: Alicization - War of Underworld,7.63,Action,Sword Art Online Alicization War of Underworld,ソードアート・オンライン アリシゼーション War of Underworld,TV,12,"Oct 13, 2019 to Dec 29, 2019",Fall 2019,"Aniplex, Bandai Namco Entertainment, Egg Firm,...",Aniplex of America,A-1 Pictures,Light novel,23 min. per ep.,R - 17+ (violence & profanity),1204.0,308,428786,4366,32060,310819,5416,5120,75371,9.0,1,1.000000,1
4,3783,21,1557,8,2,1557,Shounen Onmyouji,7.59,Action,Shonen Onmyoji,少年陰陽師,TV,26,"Oct 3, 2006 to Mar 13, 2007",Fall 2006,"Sotsu, Marine Entertainment","Funimation, Geneon Entertainment USA",Studio Deen,Light novel,23 min. per ep.,PG-13 - Teens 13 or older,1298.0,2293,48538,307,3025,22739,2497,2264,18013,7.5,1,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55886,62392029,353365,23233,9,2,23233,Shinmai Maou no Testament,6.8,Action,The Testament of Sister New Devil,新妹魔王の契約者〈テスタメント〉,TV,12,"Jan 8, 2015 to Mar 26, 2015",Winter 2015,"Kadokawa Shoten, AT-X, Nippon Columbia, Kadoka...","Funimation, Crunchyroll",Production IMS,Light novel,23 min. per ep.,R+ - Mild Nudity,4428.0,261,476324,1660,24554,347969,10319,20676,72806,9.0,1,1.000000,1
55887,62392756,353369,32949,9,2,32949,Kuzu no Honkai,7.31,Drama,Scum's Wish,クズの本懐,TV,12,"Jan 13, 2017 to Mar 31, 2017",Winter 2017,"Aniplex, Dentsu, Square Enix, Studio Hibari, F...",Sentai Filmworks,Lerche,Manga,22 min. per ep.,R+ - Mild Nudity,2294.0,272,462466,4913,33590,264613,13307,24148,126808,9.0,1,1.000000,1
55888,62392696,353369,17895,9,2,17895,Golden Time,7.78,Comedy,Golden Time,ゴールデンタイム,TV,24,"Oct 4, 2013 to Mar 28, 2014",Fall 2013,"Genco, Starchild Records",Sentai Filmworks,J.C.Staff,Light novel,24 min. per ep.,PG-13 - Teens 13 or older,855.0,138,688649,9875,39602,442612,25435,30567,150433,9.0,1,1.000000,1
55889,62394165,353383,6114,9,2,6114,Rainbow: Nisha Rokubou no Shichinin,8.52,Drama,Rainbow,RAINBOW 二舎六房の七人,TV,26,"Apr 7, 2010 to Sep 29, 2010",Spring 2010,"VAP, Nippon Television Network",Funimation,Madhouse,Manga,22 min. per ep.,R - 17+ (violence & profanity),94.0,466,304557,7421,20996,135582,16102,12168,119709,9.0,1,1.000000,1


In [11]:
merged['Aired'] = merged['Aired'].str.extract('(\d\d\d\d)')
merged['Aired'] = merged['Aired'].apply(lambda x: x[:3] + '0')
merged.rename(columns={'Rating':'Level'}, inplace=True)

In [12]:
# merged.Score = merged.Score.astype(int)

In [13]:
# CATEGORICAL : watching status -> int type --> ERROR
merged['watching_status'] = merged['watching_status'].astype(str)
# y_label -1
merged['rating'] = merged['rating'] - 1


In [14]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55891 entries, 0 to 55890
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       55891 non-null  int64  
 1   user_id          55891 non-null  int64  
 2   anime_id         55891 non-null  int64  
 3   rating           55891 non-null  int64  
 4   watching_status  55891 non-null  object 
 5   MAL_ID           55891 non-null  int64  
 6   Name             55891 non-null  object 
 7   Score            55891 non-null  object 
 8   Genres           55891 non-null  object 
 9   English name     55891 non-null  object 
 10  Japanese name    55891 non-null  object 
 11  Type             55891 non-null  object 
 12  Episodes         55891 non-null  int64  
 13  Aired            55891 non-null  object 
 14  Premiered        55891 non-null  object 
 15  Producers        55891 non-null  object 
 16  Licensors        55891 non-null  object 
 17  Studios     

In [15]:
#Categorical base columns
genres = tf.contrib.layers.sparse_column_with_keys(column_name="Genres", keys=['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia',
       'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life',
       'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Unknown',
       'Vampire', 'Yaoi', 'Yuri'])
watching_status = tf.contrib.layers.sparse_column_with_keys(column_name="watching_status", keys=['0','1','2','3','4','5'])
level = tf.contrib.layers.sparse_column_with_keys(column_name="Level", keys=['All ages','Children','PG13','R-17','Nudity','Hentai'])
aired = tf.contrib.layers.sparse_column_with_keys(column_name="Aired", keys=['2010', '2000', '2020', '1980', '1990', '1970', '1960'])
studios = tf.contrib.layers.sparse_column_with_hash_bucket("Studios", hash_bucket_size=50)
source = tf.contrib.layers.sparse_column_with_keys(column_name='Source', keys=['Light novel', 'Manga', 'Game', '4-koma manga', 'Web manga',
       'Other', 'Original', 'Visual novel', 'Novel', 'Book', 'Card game',
       'Music', 'Digital manga'])

# Continuous base columns.
episodes = tf.contrib.layers.real_valued_column("Episodes")
watch_rate = tf.contrib.layers.real_valued_column("watch_rate")


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [16]:
wide_columns = [
  genres, studios, aired, source,
  tf.contrib.layers.crossed_column([genres, watching_status], hash_bucket_size=int(1e4))]

In [17]:
deep_columns = [
  tf.contrib.layers.embedding_column(genres, dimension=8),
  tf.contrib.layers.embedding_column(watching_status, dimension=8),
  tf.contrib.layers.embedding_column(level, dimension=8),
  tf.contrib.layers.embedding_column(aired, dimension=8),
  tf.contrib.layers.embedding_column(studios, dimension=8),
  tf.contrib.layers.embedding_column(source, dimension=8),
  episodes, watch_rate]

In [30]:
#import tempfile
#model_dir = tempfile.mkdtemp()

m = tf.contrib.learn.DNNLinearCombinedClassifier(
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    n_classes=10,
    dnn_hidden_units=[100, 50])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1f548ee50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmp8_qw4tqe', '_session_creation_timeout_secs': 7200}


#Training and Evaluating the model


In [19]:
check.columns

NameError: ignored

In [31]:
#import pandas as pd
#import urllib

# Define the column names for the data sets.
COLUMNS = ['user_id','anime_id','rating','watching_status','watch_rate','Genres','English name','Episodes','Level','Members',
           'Watching','Completed','On-Hold','Dropped','Plan to Watch', 'Aired','Source','Studios','avg_rating']

LABEL_COLUMN = 'rating'
CATEGORICAL_COLUMNS = ['watching_status',  'Level','Genres','Aired','Source','Studios' ]
CONTINUOUS_COLUMNS = ['Episodes','Members','Watching','Completed',
             'On-Hold','Dropped','Plan to Watch','avg_rating','watch_rate']

# Download the training and test data to temporary files.
# Alternatively, you can download them yourself and change train_file and
# test_file to your own paths.
#train_file = tempfile.NamedTemporaryFile()
#test_file = tempfile.NamedTemporaryFile()
#urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
#urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# Divide the training and test data sets
df_train = merged[COLUMNS].iloc[:44712]
df_test = merged[COLUMNS].iloc[44712:]


def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k) to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)], values=df[k].values, dense_shape=[df[k].size, 1]) for k in CATEGORICAL_COLUMNS}
                      
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols.items() | categorical_cols.items())
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)

In [21]:
m.fit(input_fn=train_input_fn, steps=200)

results = m.evaluate(input_fn=eval_input_fn, steps=1)

for key in sorted(results):
  print('{} : {}'.format(key, results[key]))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:te

In [22]:
def input_fn_predict(df):
  return input_fn(df)

In [23]:
def input_fn_predict():
  return input_fn(df_test)
  pass
m.fit(input_fn=train_input_fn, steps=100)
metrics = m.evaluate(input_fn=eval_input_fn, steps=10)
predictions = m.predict(input_fn=input_fn_predict)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjk5e1256/model.ckpt-202
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 202 into /tmp/tmpjk5e1256/model.ckpt.
INFO:tensorflow:loss = 1.8388011, step = 204
INFO:tensorflow:Saving checkpoints for 304 into /tmp/tmpjk5e1256/model.ckpt.
INFO:tensorflow:Loss for final step: 1.8321763.
INFO:tensorflow:Starting evaluation at 2021-09-22T06:14:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjk5e1256/model.ckpt-304
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evalu

In [24]:
print(predictions)

<generator object _as_iterable at 0x7fa1ef8602d0>
